### Introduction

In [1]:
#!git clone https://github.com/myshell-ai/OpenVoice

In [2]:
# pip install openvino

# todo: unfreeze dependencies
# %pip install -q --extra-index-url https://download.pytorch.org/whl/cpu  "torch>=2.1.0" "torchaudio>=2.1.0"
# wavmark==0.0.2 also installs torch

In [3]:
# %pip install librosa==0.9.1 \
# faster-whisper==0.9.0 \
# pydub==0.25.1 \
# whisper-timestamped==1.14.2 \
# tqdm \
# inflect==7.0.0 \
# unidecode==1.3.7 \
# eng_to_ipa==0.0.2 \
# wavmark==0.0.2 \
# pypinyin==0.50.0 \
# cn2an==0.5.22 \
# jieba==0.42.1 \
# langid==1.1.6
# gradio==3.48.0 \

In [4]:
# %pip install gradio==3.48.0

In [5]:
import openvino as ov
import os
import torch
from openvoice_utils import get_tts_forward, get_converter_forward, OVOpenVoiceTTS, OVOpenVoiceConvert

In [6]:
%cd OpenVoice

/home/epavel/devel/openvino_notebooks/notebooks/408-openvoice/OpenVoice


/home/epavel/devel/openvino_notebooks/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [7]:
# !mkdir -p checkpoints/converter/
# !mkdir -p checkpoints/base_speakers/EN/
# !mkdir -p checkpoints/base_speakers/ZH/

# !wget https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints/converter/checkpoint.pth -O checkpoints/converter/checkpoint.pth
# !wget https://huggingface.co/myshell-ai/OpenVoice/raw/main/checkpoints/converter/config.json -O checkpoints/converter/config.json

# !wget https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints/base_speakers/EN/checkpoint.pth -O checkpoints/base_speakers/EN/checkpoint.pth
# !wget https://huggingface.co/myshell-ai/OpenVoice/raw/main/checkpoints/base_speakers/EN/config.json -O checkpoints/base_speakers/EN/config.json

# !wget https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints/base_speakers/ZH/checkpoint.pth -O checkpoints/base_speakers/ZH/checkpoint.pth
# !wget https://huggingface.co/myshell-ai/OpenVoice/raw/main/checkpoints/base_speakers/ZH/config.json -O checkpoints/base_speakers/ZH/config.json

# !wget https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints/base_speakers/EN/en_default_se.pth -O checkpoints/base_speakers/EN/en_default_se.pth
# !wget https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints/base_speakers/EN/en_style_se.pth -O checkpoints/base_speakers/EN/en_style_se.pth
# !wget https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints/base_speakers/ZH/zh_default_se.pth -O checkpoints/base_speakers/ZH/zh_default_se.pth

--2024-01-22 18:47:29--  https://huggingface.co/myshell-ai/OpenVoice/resolve/main/checkpoints/base_speakers/ZH/checkpoint.pth
Resolving proxy-dmz.intel.com (proxy-dmz.intel.com)... 10.102.248.16
Connecting to proxy-dmz.intel.com (proxy-dmz.intel.com)|10.102.248.16|:912... connected.
Proxy request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/c4/4f/c44ff1065a97d8c91e31c6989e0b1f15abb8c70de9951f7f5b9adda9a9c3a4f5/de9fb0eb749f3254130fe0172fcbb20e75f88a9b16b54dd0b73cac0dc40da7d9?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27checkpoint.pth%3B+filename%3D%22checkpoint.pth%22%3B&Expires=1706204855&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNjIwNDg1NX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2M0LzRmL2M0NGZmMTA2NWE5N2Q4YzkxZTMxYzY5ODllMGIxZjE1YWJiOGM3MGRlOTk1MWY3ZjViOWFkZGE5YTljM2E0ZjUvZGU5ZmIwZWI3NDlmMzI1NDEzMGZlMDE3MmZjYmIyMGU3NWY4OGE5YjE2YjU0ZGQwYjcz

In [8]:
import se_extractor
from api import BaseSpeakerTTS, ToneColorConverter

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [9]:
en_ckpt_base = 'checkpoints/base_speakers/EN'
zh_ckpt_base = 'checkpoints/base_speakers/ZH'

device="cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs'

en_base_speaker_tts = BaseSpeakerTTS(f'{en_ckpt_base}/config.json', device=device)
en_base_speaker_tts.load_ckpt(f'{en_ckpt_base}/checkpoint.pth')

zh_base_speaker_tts = BaseSpeakerTTS(f'{zh_ckpt_base}/config.json', device=device)
zh_base_speaker_tts.load_ckpt(f'{zh_ckpt_base}/checkpoint.pth')

ckpt_converter = 'checkpoints/converter'
tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

Loaded checkpoint 'checkpoints/base_speakers/EN/checkpoint.pth'
missing/unexpected keys: [] []
Loaded checkpoint 'checkpoints/base_speakers/ZH/checkpoint.pth'
missing/unexpected keys: [] []
Loaded checkpoint 'checkpoints/converter/checkpoint.pth'
missing/unexpected keys: [] []


### Convert models to OpenVINO

In [12]:
en_tts_model = OVOpenVoiceTTS(en_base_speaker_tts, ir_path='en_openvoice_tts.xml')
zh_tts_model = OVOpenVoiceTTS(en_base_speaker_tts, ir_path='zh_openvoice_tts.xml')
color_convert_model = OVOpenVoiceConvert(tone_color_converter, ir_path='openvoice_converter.xml')

en_tts_model.compile()
zh_tts_model.compile()
color_convert_model.compile()

In [13]:
# load speaker embeddings
en_source_default_se = torch.load(f'{en_ckpt_base}/en_default_se.pth').to(device)
en_source_style_se = torch.load(f'{en_ckpt_base}/en_style_se.pth').to(device)
zh_source_se = torch.load(f'{zh_ckpt_base}/zh_default_se.pth').to(device)

# source_se = torch.load(f'{ckpt_base}/en_default_se.pth').to(device)

# need to install ffmpeg in the system
reference_speaker = 'resources/example_reference.mp3'
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, target_dir='processed', vad=True)

### Inference

In [14]:
# %pip install gradio==4.15
from openvoice_gradio import get_demo

demo = get_demo(output_dir, color_convert_model, en_tts_model, zh_tts_model, en_source_default_se, en_source_style_se, zh_source_se)
demo.queue(max_size=2)
demo.launch(server_name="0.0.0.0", server_port=7860)

/home/epavel/devel/openvino_notebooks/.venv/lib/python3.10/site-packages/gradio/components/dropdown.py:90: UserWarning: The `max_choices` parameter is ignored when `multiselect` is False.
  warnings.warn(


Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


/home/epavel/devel/openvino_notebooks/.venv/lib/python3.10/site-packages/gradio/components/dropdown.py:90: UserWarning: The `max_choices` parameter is ignored when `multiselect` is False.
  warnings.warn(


Detected language:zh
[(0.43, 13.97), (14.51, 29.65), (29.966, 40.818), (41.038, 54.29), (55.086, 59.218), (59.662, 61.266), (62.51, 67.314), (67.438, 69.042), (69.646, 79.442), (80.334, 86.386), (86.414, 91.090875)]
after vad: dur = 85.45201814058957


Building prefix dict from the default dictionary ...


 > Text splitted to sentences.
今天天气真好, 我们一起出去吃饭吧.
 > ===========================


Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.271 seconds.
Prefix dict has been built successfully.


tʃ⁼in→tʰjɛn→tʰjɛn→tʃʰi↓ ts`⁼ən→ xɑʊ↓↑,  wo↓↑mən i↓tʃʰi↓↑ ts`ʰu→tʃʰɥ↓ ts`ʰɹ`→fan↓ p⁼a.
 length:85
 length:85


/home/epavel/devel/openvino_notebooks/.venv/lib/python3.10/site-packages/wavmark/models/my_model.py:25: UserWarning: istft will require a complex-valued input tensor in a future PyTorch release. Matching the output from stft with return_complex=True.  (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:978.)
  return torch.istft(signal_wmd_fft, n_fft=self.n_fft, hop_length=self.hop_length, window=window,


Detected language:zh
[(0.11, 6.016)]
after vad: dur = 5.906
 > Text splitted to sentences.
今天天气真好, 我们一起出去吃饭吧.
 > ===========================
tʃ⁼in→tʰjɛn→tʰjɛn→tʃʰi↓ ts`⁼ən→ xɑʊ↓↑,  wo↓↑mən i↓tʃʰi↓↑ ts`ʰu→tʃʰɥ↓ ts`ʰɹ`→fan↓ p⁼a.
 length:85
 length:85
Detected language:en
 > Text splitted to sentences.
i just wanted to check how my voice is copied
 > ===========================
aɪ dʒɪst ˈwɔntɪd tɪ tʃɛk haʊ maɪ vɔɪs ɪz ˈkɑpid.
 length:48
 length:48
Detected language:en
[(0.0, 8.05), (8.782, 12.85)]
after vad: dur = 12.118
 > Text splitted to sentences.
i just wanted to check how my voice is copied
 > ===========================
aɪ dʒɪst ˈwɔntɪd tɪ tʃɛk haʊ maɪ vɔɪs ɪz ˈkɑpid.
 length:48
 length:48
Detected language:en
 > Text splitted to sentences.
i just wanted to check how my voice is copied by this model so that i can easily make a fake voice and vocalize my diaries,
write about news, about a lot of sex and vice versa
 > ===========================
aɪ dʒɪst ˈwɔntɪd tɪ tʃɛk haʊ maɪ vɔɪs 

In [ ]:
save_path = f'{output_dir}/output_en_default.wav'

# Run the base speaker tts
text = "i am well aware of my capabilities?!! audio is generated by OpenVoice. OpenVoice can achieve multi-lingual voice cloning by simply replace the base speaker." \
"We provide an example with a Chinese base speaker here and we encourage the readers to try demo_part2.ipynb for a detailed demo." \
"Our online English classes feature lots of useful learning materials and activities to help you develop your reading skills with confidence in a safe and inclusive learning environment." \
"Practise reading with your classmates in live group classes, get reading support from a personal tutor in one-to-one lessons or practise reading by yourself at your own speed with a self-study course."

src_path = f'{output_dir}/tmp.wav'
# base_speaker_tts.tts(text, src_path, speaker='default', language='English', speed=1.0, ov_model=tts_model.compiled_model)
tts_model.tts(text, src_path, speaker='default', language='English', speed=1.0)

# Run the tone color converter
encode_message = "@MyShell"

color_convert_model.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message=encode_message)

In [ ]:
# !pip install ipywebrtc ipywidgets

from ipywebrtc import AudioRecorder, CameraStream
from IPython.display import Audio

camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

In [ ]:
from IPython.display import Audio

Audio(filename=save_path)